In [ ]:
import folium
from geopy.distance import geodesic

# Latitude and longitude for Crescent City, CA and Monterey, CA
crescent_city_coords = (41.7547, -124.2026)
monterey_coords = (36.6002, -121.8947)
cc_lat= 41.7547
m_lat= 36.6002
# Calculate the distance between the two locations
distance = geodesic(crescent_city_coords, monterey_coords).miles

# Create a folium map centered between the two locations
m = folium.Map(location=[(crescent_city_coords[0] + monterey_coords[0]) / 2, (crescent_city_coords[1] + monterey_coords[1]) / 2], zoom_start=6)

# Add markers for Crescent City and Monterey
folium.Marker(crescent_city_coords, tooltip="Crescent City, CA").add_to(m)
folium.Marker(monterey_coords, tooltip="Monterey, CA").add_to(m)

# Create a line between the two locations to show the distance
line = folium.PolyLine([crescent_city_coords, monterey_coords], color='blue', weight=2, opacity=1)
line.add_to(m)

# Display the distance as a tooltip on the line
line.add_child(folium.Tooltip(f"Distance: {distance:.2f} miles"))

# Display the map
m.save('geospatial_distance_map.html')

In [ ]:
# Installed Python equivalent of MATLAB's TTIDE to import PYTHON's UTIDE
!pip install utide
import utide
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

path_to_file = "C:\\Users\\cbeau\\Downloads\\WATER LEVELS CRESENT CITY.csv"
Tide_Crescent_City_Ca = pd.read_csv(path_to_file)

path_to_file2 = "C:\\Users\\cbeau\\Downloads\\WATER LEVELS MONTEREY.csv"
Tide_Monterey_Ca = pd.read_csv(path_to_file2)

# Separate Date Column into year, month, day columns
Tide_Crescent_City_Ca['year'] = pd.to_datetime(Tide_Crescent_City_Ca['Date']).dt.year
Tide_Crescent_City_Ca['month'] = pd.to_datetime(Tide_Crescent_City_Ca['Date']).dt.month
Tide_Crescent_City_Ca['day'] = pd.to_datetime(Tide_Crescent_City_Ca['Date']).dt.day

# Split 'Time (GMT)' into 'hour,' 'minute,' and 'second' columns
Tide_Crescent_City_Ca['hour'] = pd.to_datetime(Tide_Crescent_City_Ca['Time (GMT)']).dt.hour
Tide_Crescent_City_Ca['minute'] = pd.to_datetime(Tide_Crescent_City_Ca['Time (GMT)']).dt.minute
Tide_Crescent_City_Ca['second'] = pd.to_datetime(Tide_Crescent_City_Ca['Time (GMT)']).dt.second

# Combine 'year,' 'month,' 'day,' 'hour,' 'minute,' and 'second' into a datetime column
Tide_Crescent_City_Ca['datetime'] = pd.to_datetime(Tide_Crescent_City_Ca[['year', 'month', 'day', 'hour', 'minute', 'second']])

# Separate Date Column into year, month, day columns
Tide_Monterey_Ca['year'] = pd.to_datetime(Tide_Monterey_Ca['Date']).dt.year
Tide_Monterey_Ca['month'] = pd.to_datetime(Tide_Monterey_Ca['Date']).dt.month
Tide_Monterey_Ca['day'] = pd.to_datetime(Tide_Monterey_Ca['Date']).dt.day

# Split 'Time (GMT)' into 'hour,' 'minute,' and 'second' columns
Tide_Monterey_Ca['hour'] = pd.to_datetime(Tide_Monterey_Ca['Time (GMT)']).dt.hour
Tide_Monterey_Ca['minute'] = pd.to_datetime(Tide_Monterey_Ca['Time (GMT)']).dt.minute
Tide_Monterey_Ca['second'] = pd.to_datetime(Tide_Monterey_Ca['Time (GMT)']).dt.second

# Combine 'year,' 'month,' 'day,' 'hour,' 'minute,' and 'second' into a datetime column
Tide_Monterey_Ca['datetime'] = pd.to_datetime(Tide_Monterey_Ca[['year', 'month', 'day', 'hour', 'minute', 'second']])

# Extract relevant columns
crescent_city_data = Tide_Crescent_City_Ca[['datetime', 'Verified (ft)']]
monterey_data = Tide_Monterey_Ca[['datetime', 'Verified (ft)']]

# Detrend the water level data for Crescent City
crescent_city_data['Verified (ft)'] = crescent_city_data['Verified (ft)'] - crescent_city_data['Verified (ft)'].mean()


# Perform tidal analysis for Crescent City
crescent_city_result = utide.solve(crescent_city_data['datetime'], crescent_city_data['Verified (ft)'], lat=cc_lat)

# Detrend the water level data for Monterey
monterey_data['Verified (ft)'] = monterey_data['Verified (ft)'] - monterey_data['Verified (ft)'].mean()

# Perform tidal analysis for Monterey
monterey_result = utide.solve(monterey_data['datetime'], monterey_data['Verified (ft)'], lat=m_lat)

print ('Monterey Result', monterey_result)

In [ ]:
#######################################################################################################
# consituents for utide.solve function ['M2', 'K1', 'O1', 'S2', 'N2', 'Q1', 'NO1', 'J1', 'MSF', 'OO1',
#'2Q1', 'UPS1', 'M3', 'ETA2', 'SK3', 'MO3', 'MN4', '2SK5', 'MK3',
#'S4', '2MS6', 'M4', '2MN6', 'MS4', 'M6', '3MK7', 'M8', '2MK5', '2SM6']
#trying to identify significant constituents

# List of constituents
constituents_list = ['M2', 'K1', 'O1', 'S2', 'N2', 'Q1', 'NO1', 'J1', 'MSF', 'OO1',
                     '2Q1', 'UPS1', 'M3', 'ETA2', 'SK3', 'MO3', 'MN4', '2SK5', 'MK3',
                     'S4', '2MS6', 'M4', '2MN6', 'MS4', 'M6', '3MK7', 'M8', '2MK5', '2SM6']

# Dictionary to store reconstructed tidal signals for each constituent
reconstructed_tides = {}

# Iterate over each constituent
for constituent in constituents_list:
    # Perform tidal analysis for Monterey for the specific constituent
    monterey_result = utide.solve(monterey_data['datetime'], monterey_data['Verified (ft)'], lat=m_lat, constit=[constituent])

    # Reconstruct tidal signal for Monterey for the specific constituent
    monterey_tide = utide.reconstruct(monterey_data['datetime'], monterey_result, constit=[constituent])

    # Store the reconstructed tidal signal in the dictionary
    reconstructed_tides[constituent] = monterey_tide['h']

# Plot original water level data and reconstructed tidal signals for each constituent
plt.figure(figsize=(12, 8))
plt.plot(monterey_data['datetime'], monterey_data['Verified (ft)'], label='Original Data')

for constituent, tide_signal in reconstructed_tides.items():
    plt.plot(monterey_data['datetime'], tide_signal, label=f'Reconstructed - {constituent}')

plt.title('Water Level Data and Reconstructed Tidal Signals - Monterey')
plt.xlabel('Datetime')
plt.ylabel('Water Level (ft)')
plt.legend()
plt.show()
##########################################################################################
#trying to identify signifcant constituents with Skill Score
# Dictionary to store results for each constituent
#If SS is close to 1, it means that the variance of the residual errors is small compared to the variance of the original data, 
#indicating a good performance of the model
#skill score threshold
threshold = .1
# Initialize results_dict
results_dict = {}
# Iterate over each constituent
for constituent in constituents_list:
    # Perform tidal analysis for Monterey for the specific constituent
    monterey_result = utide.solve(monterey_data['datetime'], monterey_data['Verified (ft)'], lat=m_lat, constit=[constituent])

    # Reconstruct tidal signal for Monterey for the specific constituent
    monterey_tide = utide.reconstruct(monterey_data['datetime'], monterey_result, constit=[constituent])

    # Calculate the residual error for each constituent
    residual_error = monterey_data['Verified (ft)'] - monterey_tide['h']

    # Calculate the variance of the error for each constituent
    error_variance = np.var(residual_error)

    # Calculate the variance of the original data
    original_data_variance = np.var(monterey_data['Verified (ft)'])

    # Calculate the skill score for each constituent
    skill_score = 1 - (error_variance / original_data_variance)

    # Store the results in the dictionary
    results_dict[constituent] = {
        'Residual Error': residual_error,
        'Variance of Error': error_variance,
        'Original Data Variance': original_data_variance,
        'Skill Score': skill_score
    }

    # Print the results for each constituent
    print(f"\nConstituent: {constituent}")
    print(f"Residual Error: {residual_error}")
    print(f"Variance of Error: {error_variance:.6f}")
    print(f"Original Data Variance: {original_data_variance:.6f}")
    print(f"Skill Score: {skill_score:.6f}")

# Identify the most significant constituents based on the skill score
most_significant_constituents = [constituent for constituent, results in results_dict.items() if results['Skill Score'] > threshold]
print(f"\nMost Significant Constituents based on Skill Score: {most_significant_constituents}")
#######################################################################################
#trying toidentify significant consitutents with amplitutde to error ratios
# Dictionary to store results for each constituent
results_dict1 = {}

# Threshold for considering amplitude as significantly larger than the error

# Iterate over each constituent
for constituent in constituents_list:
    # Perform tidal analysis for Monterey for the specific constituent
    monterey_result = utide.solve(monterey_data['datetime'], monterey_data['Verified (ft)'], lat=m_lat, constit=[constituent])

    # Reconstruct tidal signal for Monterey for the specific constituent
    monterey_tide = utide.reconstruct(monterey_data['datetime'], monterey_result, constit=[constituent])

    # Calculate the residual error for each constituent
    residual_error = monterey_data['Verified (ft)'] - monterey_tide['h']

    # Calculate the amplitude, error, and amplitude-to-error ratio for each constituent
    amplitude = monterey_result['A'][0]  # Amplitude of the primary constituent
    error = monterey_result['g_ci'][0]  # Error of the primary constituent
    ratio_amplitude_to_error = amplitude / error

    # Store the results in the dictionary
    results_dict1[constituent] = {
        'Residual Error': residual_error,
        'Amplitude': amplitude,
        'Error': error,
        'Amplitude-to-Error Ratio': ratio_amplitude_to_error
    }

    # Print the results for each constituent
    print(f"\nConstituent: {constituent}")
    print(f"Residual Error: {residual_error}")
    print(f"Amplitude: {amplitude:.6f}")
    print(f"Error: {error:.6f}")
    print(f"Amplitude-to-Error Ratio: {ratio_amplitude_to_error:.6f}")

# Identify the most significant constituents based on amplitude-to-error ratio

#####################################################################################
# consituents for utide.solve function ['M2', 'K1', 'O1', 'S2', 'N2', 'Q1', 'NO1', 'J1', 'MSF', 'OO1',
#'2Q1', 'UPS1', 'M3', 'ETA2', 'SK3', 'MO3', 'MN4', '2SK5', 'MK3',
#'S4', '2MS6', 'M4', '2MN6', 'MS4', 'M6', '3MK7', 'M8', '2MK5', '2SM6']
#identified significant consitutents from the code above to be M2,K1, and 01
# reconstructing tide data from these significant constituents
constituents = ['M2', 'K1', 'O1']

# Reconstruct tidal signal for Crescent City using only M2, K1, and O1
tide_cc_reconstructed = utide.reconstruct(crescent_city_data['datetime'], crescent_city_result, constit=constituents)

# Reconstruct tidal signal for Monterey using only M2, K1, and O1
tide_m_reconstructed = utide.reconstruct(monterey_data['datetime'], monterey_result, constit=constituents)

print('Monterey Result', tide_m_reconstructed)
print('Monterey Reconstructed Tidal Signal:', tide_m_reconstructed['h'][:5])


print(tide_cc_reconstructed)

# Plotting
plt.figure(figsize=(12, 6))

plt.subplot(2, 1, 1)
plt.plot(Tide_Crescent_City_Ca['datetime'], Tide_Crescent_City_Ca['Verified (ft)'], label='Original - Crescent City')
plt.plot(Tide_Crescent_City_Ca['datetime'], tide_cc_reconstructed['h'], label='Reconstructed - Crescent City')
plt.title('Tidal Signal - Crescent City')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(Tide_Monterey_Ca['datetime'], Tide_Monterey_Ca['Verified (ft)'], label='Original - Monterey')
plt.plot(Tide_Monterey_Ca['datetime'], tide_m_reconstructed['h'], label='Reconstructed - Monterey')
plt.title('Tidal Signal - Monterey')
plt.legend()

plt.tight_layout()
plt.show()
#The residual error represents the difference between the observed data and the reconstructed tidal signal
#the variance of the error provides a measure of how much the error values vary.
# Calculate residual error and variance for Crescent City
residual_error_cc = Tide_Crescent_City_Ca['Verified (ft)'] - tide_cc_reconstructed['h']
variance_cc = np.var(residual_error_cc)

# Calculate residual error and variance for Monterey
residual_error_m = Tide_Monterey_Ca['Verified (ft)'] - tide_m_reconstructed['h']
variance_m = np.var(residual_error_m)

# Print results for Crescent City
print("\nCrescent City Results:")
print("Residual Error:", residual_error_cc)
print("Variance:", variance_cc)

# Print results for Monterey
print("\nMonterey Results:")
print("Residual Error:", residual_error_m)
print("Variance:", variance_m)
####################################################################################



#############################################################################





##########################################################################################################

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import utide

# Function to calculate truncated Fourier series matrix H and perform least-squares
def perform_least_squares(data, lat, constituents):
    N = len(data)  # Number of data points
    t = data['datetime'].values.astype(np.float64)  # Time in float64 format

    # Construct the matrix H
    H = construct_fourier_matrix(t, constituents)

    # Define the observed water level vector eta
    eta = data['Verified (ft)'].values

    # Use least-squares to find the best-fit coefficients
    coefficients, residuals, rank, s = np.linalg.lstsq(H, eta, rcond=None)

    # Calculate the inverse covariance matrix C^(-1)
    covariance_matrix_inverse = np.linalg.inv(np.dot(H.T, H))

    return coefficients, residuals, covariance_matrix_inverse

# Function to construct Fourier matrix
def construct_fourier_matrix(t, constituents):
    # Initialize an empty matrix
    H = np.empty((len(t), 0))

    # Manual definition of tidal frequencies (M2, K1, O1)
    tidal_frequencies = {'M2': 1.93227e-4, 'K1': 7.29212e-5, 'O1': 6.75977e-5}

    # Add cos and sin terms for each constituent
    for constituent in constituents:
        freq = tidal_frequencies[constituent]
        omega = 2 * np.pi / freq
        H = np.column_stack((H, np.cos(omega * t), np.sin(omega * t)))

    return H

# Latitude for Crescent City and Monterey
cc_lat = 41.7547
m_lat = 36.6002

# List of significant constituents
constituents = ['M2', 'K1', 'O1']

# Perform least-squares for Crescent City
cc_coefficients, cc_residuals, cc_covariance_matrix_inverse = perform_least_squares(crescent_city_data, cc_lat, constituents)

# Perform least-squares for Monterey
m_coefficients, m_residuals, m_covariance_matrix_inverse = perform_least_squares(monterey_data, m_lat, constituents)

# Print results for Crescent City
print('Crescent City Results:')
print('Best-fit coefficients:', cc_coefficients)
print('Residuals:', cc_residuals)
print('Inverse Covariance Matrix (C^(-1)):', cc_covariance_matrix_inverse)

# Print results for Monterey
print('\nMonterey Results:')
print('Best-fit coefficients:', m_coefficients)
print('Residuals:', m_residuals)
print('Inverse Covariance Matrix (C^(-1)):', m_covariance_matrix_inverse)
